# Análisis de Frecuencias del Micrófono - Python
Función para capturar y analizar frecuencias en tiempo real desde el micrófono

## 1. Importar Librerías Requeridas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import sounddevice as sd
import threading
import time
from collections import deque

print("Librerías importadas correctamente")

## 2. Capturar Audio del Micrófono

In [ ]:
def capture_audio_chunk(duration=1.0, sample_rate=44100):
    """
    Captura un fragmento de audio del micrófono.
    
    Args:
        duration: Duración de captura en segundos (default: 1.0)
        sample_rate: Frecuencia de muestreo en Hz (default: 44100)
    
    Returns:
        numpy array con los datos de audio
    """
    print(f"Capturando audio durante {duration} segundos...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()
    return audio_data.flatten(), sample_rate

# Prueba rápida
print("Función de captura lista")

## 3. Convertir Audio al Dominio de Frecuencias (FFT)

In [ ]:
def compute_fft(audio_data, sample_rate):
    """
    Calcula la Transformada Rápida de Fourier (FFT) del audio.
    
    Args:
        audio_data: Array de datos de audio
        sample_rate: Frecuencia de muestreo
    
    Returns:
        frequencies: Array de frecuencias
        magnitudes: Array de magnitudes (amplitud)
    """
    # Aplicar ventana de Hamming para reducir artefactos
    windowed = audio_data * np.hamming(len(audio_data))
    
    # Calcular FFT
    fft_result = fft(windowed)
    magnitudes = np.abs(fft_result)
    
    # Obtener solo la mitad positiva del espectro
    magnitudes = magnitudes[:len(magnitudes)//2]
    
    # Calcular frecuencias
    frequencies = fftfreq(len(windowed), 1/sample_rate)[:len(magnitudes)]
    
    return frequencies, magnitudes

print("Función FFT lista")

## 4. Analizar Espectro de Frecuencias

In [ ]:
def analyze_frequencies(frequencies, magnitudes, top_n=5):
    """
    Analiza el espectro de frecuencias.
    
    Args:
        frequencies: Array de frecuencias
        magnitudes: Array de magnitudes
        top_n: Número de frecuencias dominantes a retornar
    
    Returns:
        dict con análisis de frecuencias
    """
    # Frecuencia dominante
    dominant_idx = np.argmax(magnitudes)
    dominant_freq = frequencies[dominant_idx]
    dominant_mag = magnitudes[dominant_idx]
    
    # Top N frecuencias
    top_indices = np.argsort(magnitudes)[-top_n:][::-1]
    top_freqs = frequencies[top_indices]
    top_mags = magnitudes[top_indices]
    
    # Estadísticas
    max_mag = np.max(magnitudes)
    min_mag = np.min(magnitudes)
    mean_mag = np.mean(magnitudes)
    
    return {
        'dominant_frequency': dominant_freq,
        'dominant_magnitude': dominant_mag,
        'top_frequencies': list(zip(top_freqs, top_mags)),
        'max_magnitude': max_mag,
        'min_magnitude': min_mag,
        'mean_magnitude': mean_mag
    }

print("Función de análisis lista")

## 5. Visualizar Análisis de Frecuencias

In [ ]:
def plot_frequency_spectrum(frequencies, magnitudes, audio_data, sample_rate):
    """
    Visualiza el espectro de frecuencias y la forma de onda.
    
    Args:
        frequencies: Array de frecuencias
        magnitudes: Array de magnitudes
        audio_data: Datos de audio original
        sample_rate: Frecuencia de muestreo
    """
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Gráfico 1: Forma de onda
    time_axis = np.linspace(0, len(audio_data)/sample_rate, len(audio_data))
    axes[0].plot(time_axis, audio_data, linewidth=0.5, color='blue')
    axes[0].set_xlabel('Tiempo (s)')
    axes[0].set_ylabel('Amplitud')
    axes[0].set_title('Forma de Onda - Dominio del Tiempo')
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Espectro de frecuencias
    axes[1].plot(frequencies, magnitudes, linewidth=1, color='green')
    axes[1].set_xlabel('Frecuencia (Hz)')
    axes[1].set_ylabel('Magnitud')
    axes[1].set_title('Espectro de Frecuencias - Dominio de Frecuencia')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xlim([0, 5000])  # Limitar a 5000 Hz para mejor visualización
    
    plt.tight_layout()
    plt.show()

print("Función de visualización lista")

## 6. Función Completa de Análisis

In [ ]:
def analyze_microphone_frequencies(duration=2.0, sample_rate=44100, visualize=True):
    """
    Función principal que captura y analiza frecuencias del micrófono.
    
    Args:
        duration: Duración de captura en segundos (default: 2.0)
        sample_rate: Frecuencia de muestreo en Hz (default: 44100)
        visualize: Si mostrar gráficos (default: True)
    
    Returns:
        dict con resultados del análisis
    """
    print(f"\n=== ANALIZADOR DE FRECUENCIAS DEL MICRÓFONO ===")
    print(f"Parámetros: Duración={duration}s, Sample Rate={sample_rate}Hz\n")
    
    # Paso 1: Capturar audio
    audio_data, sr = capture_audio_chunk(duration, sample_rate)
    
    # Paso 2: Calcular FFT
    frequencies, magnitudes = compute_fft(audio_data, sr)
    
    # Paso 3: Analizar
    analysis = analyze_frequencies(frequencies, magnitudes, top_n=5)
    
    # Mostrar resultados
    print(f"Frecuencia Dominante: {analysis['dominant_frequency']:.2f} Hz (Magnitud: {analysis['dominant_magnitude']:.2f})")
    print(f"\nTop 5 Frecuencias:")
    for i, (freq, mag) in enumerate(analysis['top_frequencies'], 1):
        print(f"  {i}. {freq:.2f} Hz - Magnitud: {mag:.2f}")
    
    print(f"\nEstadísticas:")
    print(f"  Magnitud Máxima: {analysis['max_magnitude']:.2f}")
    print(f"  Magnitud Mínima: {analysis['min_magnitude']:.2f}")
    print(f"  Magnitud Promedio: {analysis['mean_magnitude']:.2f}")
    
    # Paso 4: Visualizar
    if visualize:
        plot_frequency_spectrum(frequencies, magnitudes, audio_data, sr)
    
    return analysis

print("Función principal de análisis lista")

In [ ]:
## 7. Extraer Secuencia de Frecuencias (Máximo 2 elementos)

In [ ]:
def get_top_frequencies(audio_data, sample_rate, num_frequencies=2, threshold=20):
    """
    Extrae las frecuencias dominantes del audio.
    
    Args:
        audio_data: Array de datos de audio
        sample_rate: Frecuencia de muestreo
        num_frequencies: Número de frecuencias a retornar (default: 2)
        threshold: Umbral mínimo de magnitud (default: 20)
    
    Returns:
        list: Array con máximo N diccionarios {'frequency': Hz, 'magnitude': valor}
    """
    # Calcular FFT
    frequencies, magnitudes = compute_fft(audio_data, sample_rate)
    
    # Crear array de objetos con frecuencia y magnitud
    freq_data = [
        {'frequency': int(freq), 'magnitude': mag}
        for freq, mag in zip(frequencies, magnitudes)
        if mag > threshold
    ]
    
    if not freq_data:
        return []
    
    # Ordenar por magnitud descendente y tomar máximo N
    top_freqs = sorted(freq_data, key=lambda x: x['magnitude'], reverse=True)[:num_frequencies]
    
    return top_freqs


def analyze_frequency_sequence(audio_data, sample_rate, num_frequencies=2, threshold=20):
    """
    Analiza frecuencias y devuelve secuencia de máximo N frecuencias dominantes.
    
    Args:
        audio_data: Array de datos de audio
        sample_rate: Frecuencia de muestreo
        num_frequencies: Número máximo de frecuencias (default: 2)
        threshold: Umbral mínimo de magnitud (default: 20)
    
    Returns:
        list: Array con máximo N frecuencias [freq1, freq2] o menos
    """
    # Calcular FFT
    frequencies, magnitudes = compute_fft(audio_data, sample_rate)
    
    # Detectar picos locales (máximos relativos)
    peaks = []
    for i in range(1, len(magnitudes) - 1):
        if (magnitudes[i] > magnitudes[i-1] and 
            magnitudes[i] > magnitudes[i+1] and 
            magnitudes[i] > threshold):
            peaks.append({
                'frequency': int(frequencies[i]),
                'magnitude': magnitudes[i],
                'index': i
            })
    
    if not peaks:
        return []
    
    # Ordenar por magnitud y tomar máximo N
    top_peaks = sorted(peaks, key=lambda x: x['magnitude'], reverse=True)[:num_frequencies]
    
    # Retornar solo las frecuencias
    return [p['frequency'] for p in top_peaks]


print("Funciones de extracción de frecuencias implementadas")

## Ejemplos de Uso

In [ ]:
# Ejemplo 1: Capturar audio y obtener top 2 frecuencias con magnitud
print("=== EJEMPLO 1: get_top_frequencies() ===\n")

# audio_data, sr = capture_audio_chunk(duration=2.0, sample_rate=44100)
# top_freqs = get_top_frequencies(audio_data, sr, num_frequencies=2)

# print("Top 2 Frecuencias con magnitud:")
# for i, freq_data in enumerate(top_freqs, 1):
#     print(f"  {i}. Frecuencia: {freq_data['frequency']} Hz - Magnitud: {freq_data['magnitude']:.2f}")

print("Descomenta el código anterior para ejecutar")
print("\n" + "="*50 + "\n")

In [ ]:
# Ejemplo 2: Obtener secuencia de máximo 2 frecuencias (sin magnitud)
print("=== EJEMPLO 2: analyze_frequency_sequence() ===\n")

# audio_data, sr = capture_audio_chunk(duration=2.0, sample_rate=44100)
# frequency_sequence = analyze_frequency_sequence(audio_data, sr, num_frequencies=2)

# print("Secuencia de frecuencias (máximo 2):")
# print(f"  {frequency_sequence}")
# print(f"  Tipo: {type(frequency_sequence)}")
# print(f"  Cantidad de frecuencias detectadas: {len(frequency_sequence)}")

print("Descomenta el código anterior para ejecutar")

## Uso: Ejecutar Análisis

In [ ]:
# Ejecutar análisis (captura 2 segundos de audio)
# DESCOMENTA LA LÍNEA SIGUIENTE PARA USAR:
# results = analyze_microphone_frequencies(duration=2.0, sample_rate=44100, visualize=True)

## Monitoreo Continuo (Opcional)

In [ ]:
def continuous_frequency_monitor(duration_total=10, chunk_duration=1.0, sample_rate=44100):
    """
    Monitorea frecuencias continuamente en tiempo real.
    
    Args:
        duration_total: Duración total del monitoreo en segundos
        chunk_duration: Duración de cada análisis en segundos
        sample_rate: Frecuencia de muestreo
    """
    print(f"\n=== MONITOREO CONTINUO - {duration_total}s ===")
    num_chunks = int(duration_total / chunk_duration)
    
    for chunk_num in range(num_chunks):
        print(f"\nChunk {chunk_num + 1}/{num_chunks}:")
        audio_data, sr = capture_audio_chunk(chunk_duration, sample_rate)
        frequencies, magnitudes = compute_fft(audio_data, sr)
        analysis = analyze_frequencies(frequencies, magnitudes)
        print(f"  Frecuencia Dominante: {analysis['dominant_frequency']:.2f} Hz")

# DESCOMENTA PARA USAR:
# continuous_frequency_monitor(duration_total=5, chunk_duration=1.0)